In [1]:
from sklearn.preprocessing import normalize, LabelEncoder
from patsy import dmatrix
from sklearn.linear_model import LinearRegression, LassoCV, LassoLarsCV, RidgeCV, ElasticNetCV, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn import cross_validation
from sklearn.metrics import explained_variance_score, mean_squared_error, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from datetime import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import xgboost as xgb
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
bat_df = pd.read_csv('./[ST]Personal_Project/dataset/mlb_fa_bat_list2.csv')
test_bat = pd.read_csv('./[ST]Personal_Project/dataset/fa_2018_bat_list.csv')

In [4]:
bat_df.shape, test_bat.shape

((660, 40), (11, 40))

In [5]:
bat_df

,Name,Yr,Team,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,BB%,K%,BB/K,BABIP,HBP,SF,SH,GDP,SB,CS,Def,ISO,wOBA,wRAA,AVG,OBP,SLG,OPS,WAR,wRC+,WPA,Contract Yrs,Total Salary($),Avg Salary($)
0,Aaron Boone,2005,Indians,143,511,565,124,88,19,1,16,61,60,35,3,92,6.2,16.3,0.38,0.264,9,6,4,16,9,3,-2.0,0.135,0.296,-14.1,0.243,0.299,0.378,0.677,0.2,80.0,-2.22,1.0,925000.0,9.250000e+05
1,Adam Kennedy,2005,Angels,129,416,460,125,100,23,0,2,49,37,29,1,64,6.3,13.9,0.45,0.348,7,3,5,5,19,4,15.5,0.070,0.323,-1.1,0.300,0.354,0.370,0.724,3.2,98.0,0.82,3.0,10000000.0,3.333333e+06
2,Alex Cora,2005,Red Sox,96,250,273,58,43,8,4,3,25,24,11,0,30,4.0,11.0,0.37,0.250,5,3,4,6,7,2,11.0,0.100,0.268,-13.1,0.232,0.275,0.332,0.607,0.7,59.0,-1.51,2.0,4000000.0,2.000000e+06
3,Alex Gonzalez,2005,Marlins,130,435,478,115,80,30,1,9,47,45,31,10,81,6.8,19.3,0.38,0.316,5,3,4,13,5,3,11.8,0.140,0.320,-1.9,0.269,0.323,0.410,0.733,2.1,96.0,0.30,3.0,14000000.0,4.666667e+06
4,Alfonso Soriano,2005,Rangers,156,637,682,171,90,43,2,36,102,104,33,3,125,4.8,18.3,0.26,0.281,7,5,0,6,30,2,-13.6,0.243,0.347,12.2,0.268,0.309,0.512,0.821,2.4,108.0,1.45,8.0,136000000.0,1.700000e+07
5,Andy Pettitte,2005,Astros,33,62,79,5,5,0,0,0,1,3,1,0,19,1.3,24.1,0.05,0.114,0,1,15,0,0,0,8.0,0.000,0.081,-16.1,0.081,0.094,0.081,0.174,-0.6,-66.0,-1.32,1.0,16000000.0,1.600000e+07
6,Aramis Ramirez,2005,Cubs,123,463,506,140,79,30,0,31,72,92,35,4,60,6.9,11.9,0.58,0.291,6,2,0,15,0,1,-2.6,0.266,0.392,27.5,0.302,0.358,0.568,0.926,2.9,136.0,2.99,5.0,75000000.0,1.500000e+07
7,Aubrey Huff,2005,Devil Rays,154,575,636,150,100,26,2,22,70,92,49,13,88,7.7,13.8,0.56,0.271,5,7,0,12,8,7,-16.2,0.167,0.318,-4.2,0.261,0.321,0.428,0.749,0.2,94.0,0.12,3.0,20000000.0,6.666667e+06
8,Bengie Molina,2005,Angels,119,410,449,121,89,17,0,15,45,69,27,2,41,6.0,9.1,0.66,0.294,1,6,5,14,0,2,5.3,0.151,0.337,4.2,0.295,0.336,0.446,0.782,1.7,108.0,1.01,3.0,16000000.0,5.333333e+06
9,Carlos Lee,2005,Brewers,162,618,688,164,91,41,0,32,85,114,57,7,87,8.3,12.6,0.66,0.259,2,11,0,8,13,4,-9.7,0.222,0.343,9.7,0.265,0.324,0.487,0.811,1.8,106.0,1.17,6.0,100000000.0,1.666667e+07


In [14]:
test_bat = test_bat.rename(columns={'Contract_Yrs' : 'Contract Yrs', 'Total_Salary' : 'Total Salary($)',
                                    'Salary' : 'Avg Salary($)'})

In [16]:
bat_df.loc[173, 'Team'] = 'Nationals'
bat_df.loc[175, 'Team'] = 'Brewers'
bat_df.loc[182, 'Team'] = 'Pirates'
bat_df.loc[186, 'Team'] = 'Indians'
bat_df.loc[192, 'Team'] = 'Reds'
bat_df.loc[225, 'Team'] = 'Pirates'
bat_df.loc[572, 'Team'] = 'Phillies'

In [17]:
merged_bat = pd.concat([bat_df, test_bat])

merge_1 = merged_bat.set_index('Name', inplace=False)
merge_2 = merge_1.reset_index()

merged_bat = merge_2

In [18]:
print(bat_df.columns.tolist())

['Name', 'Yr', 'Team', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'BB%', 'K%', 'BB/K', 'BABIP', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'Def', 'ISO', 'wOBA', 'wRAA', 'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'wRC+', 'WPA', 'Contract Yrs', 'Total Salary($)', 'Avg Salary($)']


In [19]:
to_list = ['Name', 'Yr', 'Team', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'BB%', 'K%',
           'BB/K', 'BABIP', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'Def', 'ISO', 'wOBA', 'wRAA', 'AVG', 'OBP', 'SLG', 'OPS',
           'WAR', 'wRC+', 'WPA', 'Contract Yrs', 'Total Salary($)', 'Avg Salary($)']
merged_bat = merged_bat[to_list]

In [20]:
merged_bat.tail(30)

,Name,Yr,Team,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,BB%,K%,BB/K,BABIP,HBP,SF,SH,GDP,SB,CS,Def,ISO,wOBA,wRAA,AVG,OBP,SLG,OPS,WAR,wRC+,WPA,Contract Yrs,Total Salary($),Avg Salary($)
641,Jose Bautista,2016,Blue Jays,116,423,517,99,52,24,1,22,68,69,87,1,103,16.8,19.9,0.84,0.255,3,4,0,21,2,2,-12.4,0.217,0.355,15.5,0.234,0.366,0.452,0.817,1.4,122.0,1.11,1.0,18500000.0,1.850000e+07
642,Josh Reddick,2016,Dodgers,115,398,439,112,84,17,1,10,53,37,39,5,56,8.9,12.8,0.70,0.306,0,1,0,8,8,3,-6.3,0.123,0.324,2.0,0.281,0.345,0.405,0.749,1.1,104.0,0.25,4.0,52000000.0,1.300000e+07
643,Justin Turner,2016,Dodgers,151,556,622,153,89,34,3,27,79,90,48,1,107,7.7,17.2,0.45,0.293,10,8,0,16,4,1,16.0,0.218,0.353,17.9,0.275,0.339,0.493,0.832,5.5,123.0,2.87,4.0,64000000.0,1.600000e+07
644,Kendrys Morales,2016,Royals,154,558,618,147,93,24,0,30,65,93,48,2,120,7.8,19.4,0.40,0.283,7,5,0,20,0,0,-16.3,0.204,0.339,10.6,0.263,0.327,0.468,0.795,0.8,111.0,-0.58,3.0,33000000.0,1.100000e+07
645,Kurt Suzuki,2016,Twins,106,345,373,89,56,24,1,8,34,49,18,0,48,4.8,12.9,0.38,0.276,5,4,1,9,0,0,3.4,0.145,0.303,-4.6,0.258,0.301,0.403,0.704,0.5,85.0,-1.36,1.0,1500000.0,1.500000e+06
646,Luis Valbuena,2016,Astros,90,292,342,76,45,17,1,13,38,40,44,2,81,12.9,23.7,0.54,0.315,1,2,3,5,1,1,-1.3,0.199,0.351,9.2,0.260,0.357,0.459,0.816,2.0,124.0,1.73,2.0,15000000.0,7.500000e+06
647,Mark Trumbo,2016,Orioles,159,613,667,157,82,27,1,47,94,108,51,1,170,7.6,25.5,0.30,0.278,3,0,0,14,2,0,-18.3,0.277,0.358,22.0,0.256,0.316,0.533,0.850,2.2,125.0,1.23,3.0,37500000.0,1.250000e+07
648,Matt Holliday,2016,Cardinals,110,382,426,94,53,20,1,20,48,62,35,1,71,8.2,16.7,0.49,0.253,8,1,0,9,0,0,-8.7,0.215,0.335,6.1,0.246,0.322,0.461,0.782,0.7,110.0,1.33,1.0,13000000.0,1.300000e+07
649,Mike Napoli,2016,Indians,150,557,645,133,76,22,1,34,92,101,78,2,194,12.1,30.1,0.40,0.296,5,5,0,15,5,1,-17.3,0.226,0.343,13.0,0.239,0.335,0.465,0.800,0.9,113.0,0.67,1.0,8500000.0,8.500000e+06
650,Mitch Moreland,2016,Rangers,147,460,503,107,64,21,0,22,49,60,35,5,118,7.0,23.5,0.30,0.266,8,0,0,8,1,0,-3.4,0.189,0.307,-4.4,0.233,0.298,0.422,0.720,0.4,86.0,-0.15,1.0,5500000.0,5.500000e+06


In [21]:
merged_bat.to_csv('./[ST]Personal_Project/dataset/merged_bat.csv', index=None)